In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
didi_df = pd.read_csv('Copy of sample ab_data - sample ab_data.csv')
didi_df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,1
3,853541,28:03.1,treatment,new_page,1
4,864975,52:26.2,control,old_page,1


In [3]:
didi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [4]:
#Check the misaligend
((didi_df.group=='control')&(didi_df.landing_page=='new_page')).sum()

1928

In [5]:
((didi_df.group=='treatment')&(didi_df.landing_page=='old_page')).sum()

1965

In [6]:
#deleting misaligend value
didi_df['misaligend'] = ((didi_df.group=='control')&(didi_df.landing_page=='new_page'))|((didi_df.group=='treatment')&(didi_df.landing_page=='old_page'))
didi_df = didi_df[-didi_df['misaligend']]

In [7]:
didi_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 290585 entries, 0 to 294477
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       290585 non-null  int64 
 1   timestamp     290585 non-null  object
 2   group         290585 non-null  object
 3   landing_page  290585 non-null  object
 4   converted     290585 non-null  int64 
 5   misaligend    290585 non-null  bool  
dtypes: bool(1), int64(2), object(3)
memory usage: 13.6+ MB


In [8]:
#checking if there any id take more than one time survey
didi_df['user_id'].value_counts().sort_values(ascending=False).head()


user_id
773192    2
834487    1
936923    1
679687    1
719014    1
Name: count, dtype: int64

In [9]:
didi_df[didi_df['user_id']==773192]

,user_id,timestamp,group,landing_page,converted,misaligend
1899,773192,37:58.8,treatment,new_page,1,False
2893,773192,55:59.6,treatment,new_page,0,False


In [10]:
didi_df.drop(1899, axis = 0,inplace = True)

In [11]:
didi_df['user_id'].value_counts().sort_values(ascending=False).head()

user_id
851104    1
834487    1
936923    1
679687    1
719014    1
Name: count, dtype: int64

In [12]:
didi_df[['user_id','group']].groupby('group').count()

,user_id
group,
control,145274
treatment,145310


In [13]:
#conversion rate of each group
didi_df[['user_id','group','converted']].groupby('group').agg({'user_id':'count','converted':'mean'})

,user_id,converted
group,,
control,145274,0.120386
treatment,145310,0.125353


In [14]:
#analyze results
#we assume that the new page does not have higher conversion rate than the old page at 5% Type I error.
#The hypothesis would be :
#H0 :  P_new - P_old <= 0
#H1 :  P_new - P_old > 0


In [15]:
convert_old =didi_df[didi_df.group=='control'].converted.sum()
convert_new = didi_df[didi_df.group=='treatment'].converted.sum()
n_old = len(didi_df[didi_df.group=='control'])
n_new= len(didi_df[didi_df.group=='treatment'])
convert_old, convert_new ,n_old, n_new

(17489, 18215, 145274, 145310)

In [16]:
conversion_dic = {'Views':{'Control':n_old,'Test':n_new},'Converts': {'Control':convert_old,'Test':convert_new}}
conversion_table = pd.DataFrame(conversion_dic)
conversion_table['Conversion %'] = conversion_table['Converts'] / conversion_table['Views']
conversion_table['Conversion %'] = conversion_table['Conversion %'].apply(lambda x: str(np.round(x,3)*100)+'%')
conversion_table

,Views,Converts,Conversion %
Control,145274,17489,12.0%
Test,145310,18215,12.5%


In [17]:
#Z-test to caculate z-value
#Formula Way
def z_test(p1,p0,n1,n0):
    delta = p1-p0
    p = (p1*n1+p0*n0)/(n1+n0)
    return delta/np.sqrt(p*(1-p)*(1/n1+1/n0))

In [18]:
p1 = convert_new/n_new
p0 = convert_old/n_old
n1 = n_new
n0 = n_old

In [19]:
z_value = z_test(p1,p0,n1,n0)
z_value

4.077481782861739

In [20]:
from scipy.stats import norm
p_value = 1-norm.cdf(z_value)
p_value


2.276304781123617e-05

In [21]:
# p-value is less than 0.05, so we can reject null hypothesis and accept alternative hypothesis that P_new-P_old >0
#the new page has higher conversion rate than the old page.


In [22]:
#python library fuction to caculate P-value
import statsmodels.api as sm

In [23]:
z_score, p_value = sm.stats.proportions_ztest([convert_new,convert_old], [n_new,n_old], alternative = 'larger')
z_score, p_value

(4.077481782861739, 2.276304781118429e-05)